In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [35]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")

In [36]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
# Preprocess the data
## Drop irrelevant features

data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [38]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [39]:
## Encode categorical variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [40]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [41]:
# OHE the 'Geography'

from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [42]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [43]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [44]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [45]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [46]:
## Combine OHE encoded column with original data

data = pd.concat([data.drop('Geography', axis = 1),geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [47]:
## Save both the encoders ans scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [48]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [49]:
## Divide the dataset into X & y
X = data.drop('Exited',axis=1)  #independent feature
y = data['Exited']  #dependent feature

#Split train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN IMPLEMENTATION

In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [52]:
(X_train.shape[1],)  # no. of input feature per sample
# number of columns (features) in first(each) row/sample.
# Keras expects a tuple as a input shape

(12,)

In [53]:
## Build our ANN Model
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # INPUT LAYER
    Dense(64, activation='relu'), # HIDDEN LAYER 1
    Dense(32, activation='relu'), # HIDDEN LAYER 2
    Dense(1, activation='sigmoid') # OUTPUT LAYER
]
)

In [54]:
model.summary()  
# Output shape: None = “any number of samples can flow through this layer”

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [55]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [56]:
## Compile the model
# Both the below ways are correct

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer= opt, loss= loss, metrics=['accuracy'])

In [57]:
## Set up the tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1) 

# It tells TensorFlow to log training details so you can visualize them in TensorBoard.
# Log histograms of weights & biases every 1 epoch

In [58]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Early stopping - stop training when model stops improving while training
# Monitor - validation lossses, pateince = 5epochs
# If validation loss does NOT improve for 5 consecutive epochs, stop training.
# After stopping, load the weights from the epoch where validation loss was lowest

In [59]:
### Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test,y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3991 - accuracy: 0.8344 - val_loss: 0.3627 - val_accuracy: 0.8550
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8556 - val_loss: 0.3439 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3472 - accuracy: 0.8605 - val_loss: 0.3452 - val_accuracy: 0.8600
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3444 - accuracy: 0.8566 - val_loss: 0.3402 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3407 - accuracy: 0.8619 - val_loss: 0.3416 - val_accuracy: 0.8620
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3408 - accuracy: 0.8602 - val_loss: 0.3499 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3351 - accuracy: 0.8630 - val_loss: 0.3507 - val_accuracy: 0.8570

In [60]:
history.history.keys()


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [61]:
model.save('model.h5')

c:\Users\Tanishq\Assigments DSAI KNaik\END TO END PROJECTS\ANN_classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [62]:
# Pickle saves Python objects; model.save() saves an entire trained neural network.
# model.save('model.h5') is designed to save Keras/TensorFlow models, not generic Python objects like StandardScaler.

In [66]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [67]:
%tensorboard --logdir logs/fit/20251226-173703

Reusing TensorBoard on port 6007 (pid 7364), started 0:00:24 ago. (Use '!kill 7364' to kill it.)

The epoch loss graph is showing us how is our epoch loss. here we can see at 13th epoch our model is overfitted, and is best generalised at 4th epoch. As we have set restore_best_weights=True means our model has automatically picked the weights from 4th epoch ;)